In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/glaucoma-detection/glaucoma.csv')
df.head()

In [ ]:
df.describe(include='all')

In [ ]:
df['Eye'].unique()

In [ ]:
df['Set'].unique()

In [ ]:
df['Glaucoma'].unique()

In [ ]:
df['eye'] = df['Eye'].map({'OD': 0, 'OS': 1})
df.head()

In [ ]:
df['set'] = df['Set'].map({'A': 0, 'B': 1})
df.head()

In [ ]:
df['image'] = np.nan
df.head()

In [ ]:
image_dir = '/kaggle/input/glaucoma-detection/Fundus_Train_Val_Data/Fundus_Scanes_Sorted'

In [ ]:
train_dir = os.path.join(image_dir, 'Train')
validation_dir = os.path.join(image_dir, 'Validation')

In [ ]:
mapping = {
    0: 'Glaucoma_Negative',
    1: 'Glaucoma_Positive'
}

In [ ]:
from skimage.io import imread
from skimage.transform import resize
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
train_image_negative = os.listdir(os.path.join(train_dir, 'Glaucoma_Negative'))
train_image_positive = os.listdir(os.path.join(train_dir, 'Glaucoma_Positive'))
print(len(train_image_negative) + len(train_image_positive))

In [ ]:
test_image_negative = os.listdir(os.path.join(validation_dir, 'Glaucoma_Negative'))
test_image_positive = os.listdir(os.path.join(validation_dir, 'Glaucoma_Positive'))
print(len(test_image_negative) + len(test_image_positive))

In [ ]:
df_train = df[df['Filename'].isin(train_image_negative) | df['Filename'].isin(train_image_positive)]
len(df_train)

In [ ]:
df_test = df[df['Filename'].isin(test_image_negative) | df['Filename'].isin(test_image_positive)]
len(df_test)

In [ ]:
df_train['Glaucoma'].value_counts()

In [ ]:
IMAGE_SIZE = (224,224)

In [ ]:
def load_image(row):
    global count
    folder_name = mapping[row['Glaucoma']]
    folder_path = os.path.join(train_dir, folder_name)
    image_path = os.path.join(folder_path, row['Filename'])
    im = imread(image_path)
    im = resize(im, IMAGE_SIZE)
    return im

In [ ]:
df_train['image'] = df_train.apply(load_image, axis=1)

In [ ]:
def load_image1(row):
    global count
    folder_name = mapping[row['Glaucoma']]
    folder_path = os.path.join(validation_dir, folder_name)
    image_path = os.path.join(folder_path, row['Filename'])
    im = imread(image_path)
    im = resize(im, IMAGE_SIZE)
    return im

In [ ]:
df_test['image'] = df_test.apply(load_image1, axis=1)

In [ ]:
X_image_train = df_train['image']
X_data_train = df_train[['ExpCDR', 'eye', 'set']]
y_train = df_train['Glaucoma']

In [ ]:
X_image_test = df_test['image']
X_data_test = df_test[['ExpCDR', 'eye', 'set']]
y_test = df_test['Glaucoma']

In [ ]:
X_image_train_stacked = np.stack(X_image_train.values)
X_image_test_stacked = np.stack(X_image_test.values)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights

In [ ]:
class_weight_dict = {
    0: class_weights[0],
    1: class_weights[1]
}
class_weight_dict

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential, layers, Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.resnet import ResNet101
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.efficientnet import EfficientNetB7
from sklearn.metrics import classification_report

In [ ]:
resnet = ResNet50(input_shape=IMAGE_SIZE+(3,), weights='imagenet', include_top=False)
vgg = VGG16(input_shape=IMAGE_SIZE+(3,), weights='imagenet', include_top=False)
xception = Xception(input_shape=IMAGE_SIZE+(3,), weights='imagenet', include_top=False)
resnet101 = ResNet101(input_shape=IMAGE_SIZE+(3,), weights='imagenet', include_top=False)
inception = InceptionV3(input_shape=IMAGE_SIZE+(3,), weights='imagenet', include_top=False)
mobilenet = MobileNetV2(input_shape=IMAGE_SIZE+(3,), weights='imagenet', include_top=False)
efficientnet = EfficientNetB7(input_shape=IMAGE_SIZE+(3,), weights='imagenet', include_top=False)

In [ ]:
model_dict = {
    'resnet50': resnet,
    'vgg16': vgg,
    'xception': xception,
    'resnet101': resnet101,
    'inception': inception,
    'mobilenet': mobilenet,
    'efficientnetB7': efficientnet
}
model_dict

In [ ]:
for key, model in model_dict.items():
    for layer in model.layers:
        layer.trainable = False

In [ ]:
for key, model in model_dict.items():
    x = layers.Flatten()(model.output)
    prediction = layers.Dense(1, activation='sigmoid')(x)
    model = Model(inputs=model.input, outputs=prediction)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    print(key)
    print(model.summary())
    model.fit(X_image_train_stacked, y_train, class_weight=class_weight_dict, epochs=10)
    y_pred = model.predict(X_image_test_stacked)
    y_pred = [round(y[0]) for y in y_pred]
    print(classification_report(y_test, y_pred))

In [ ]:
data_input = layers.Input(shape=(3,))

In [ ]:
x1 = layers.Dense(1, activation='relu')(data_input)
x1 = Model(inputs=data_input, outputs=x1)

In [ ]:
x2 = layers.Flatten()(mobilenet.output)
x2 = Model(inputs=mobilenet.input, outputs=x2)

In [ ]:
combined = layers.concatenate([x1.output, x2.output])

In [ ]:
x_combined = layers.Dense(150, activation='relu')(combined)
x_combined = layers.Dense(50, activation='relu')(x_combined)
x_combined = layers.Dense(1, activation='sigmoid')(x_combined)

In [ ]:
model = Model(inputs=[x1.input, x2.input], outputs=x_combined)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit([X_data_train, X_image_train_stacked], y=y_train, class_weight=class_weight_dict, epochs=15)

In [ ]:
model.evaluate([X_data_test, X_image_test_stacked], y_test)

In [ ]:
y_pred = model.predict([X_data_test, X_image_test_stacked])
y_pred = [round(y[0]) for y in y_pred]

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
x2_vgg = layers.Flatten()(vgg.output)
x2_vgg = Model(inputs=vgg.input, outputs=x2_vgg)

In [ ]:
combined_vgg = layers.concatenate([x1.output, x2_vgg.output])

In [ ]:
x_combined_vgg = layers.Dense(150, activation='relu')(combined_vgg)
x_combined_vgg = layers.Dense(50, activation='relu')(x_combined_vgg)
x_combined_vgg = layers.Dense(1, activation='sigmoid')(x_combined_vgg)

In [ ]:
model_vgg = Model(inputs=[x1.input, x2_vgg.input], outputs=x_combined_vgg)

In [ ]:
model_vgg.summary()

In [ ]:
model_vgg.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model_vgg.fit([X_data_train, X_image_train_stacked], y_train, class_weight=class_weight_dict, epochs=50)

In [ ]:
model_vgg.evaluate([X_data_test, X_image_test_stacked], y_test)

In [ ]:
y_pred_vgg = model_vgg.predict([X_data_test, X_image_test_stacked])
y_pred_vgg = [round(y[0]) for y in y_pred_vgg]

In [ ]:
print(classification_report(y_test, y_pred_vgg))